# Databricks AI Agent — Knowledge Assistant
An intelligent AI Agent powered by Databricks Llama-4 models that combines text reasoning and vision understanding to deliver context-rich results from multiple sources.


⚙️ Tech Stack
* Databricks Llama-4 Maverick (text reasoning)
* LangChain (agent orchestration)
* DuckDuckGo Search API
* Wikipedia API

💡 Key Features
* Zero-shot reasoning and planning with tool invocation
* Seamless integration of textual + visual information
* Structured, human-readable responses

In [0]:
%pip install --quiet --upgrade langchain ddgs langchain-community langchain-openai databricks-langchain wikipedia duckduckgo-search youtube_search langchain-databricks
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


## Tool Use / Function Calling

In [0]:
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

wikidata = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
wikidata.run("What is Generative AI?")

'Page: Generative artificial intelligence\nSummary: Generative artificial intelligence (Generative AI, GenAI, or GAI) is a subfield of artificial intelligence that uses generative models to produce text, images, videos, audio, software code or other forms of data. These models learn the underlying patterns and structures of their training data and use them to produce new data based on the input, which often comes in the form of natural language prompts.\nGenerative AI tools have become more common since the AI boom in the 2020s. This boom was made possible by improvements in transformer-based deep neural networks, particularly large language models (LLMs). Major tools include chatbots such as ChatGPT, Copilot, Gemini, Claude, Grok, and DeepSeek; text-to-image models such as Stable Diffusion, Midjourney, and DALL-E; and text-to-video models such as Veo and Sora. Technology companies developing generative AI include OpenAI, xAI, Anthropic, Meta AI, Microsoft, Google, Mistral AI, DeepSeek

In [0]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.run("What is Obama's first name?")

In [0]:
from langchain_community.tools import YouTubeSearchTool

yt_tool = YouTubeSearchTool()
yt_tool.run("The Data master")

## Agent

In [0]:
from databricks_langchain import ChatDatabricks
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_community.tools import YouTubeSearchTool
from langchain.agents import Tool,initialize_agent

In [0]:

# ---- Initialize Model from Databricks Serving Endpoint ----
chat_model = ChatDatabricks(endpoint="databricks-llama-4-maverick")

# ---- Wikipedia Tool ----
wiki_tool = Tool(
    name="Wikipedia",
    func= WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper()).run,
    description=(
        "Search Wikipedia for accurate and concise educational information "
        "on topics such as artificial intelligence, cloud computing, data engineering, "
        "and software development."
    ),
)

# ---- Youtube Tool ----
youtube_tool = Tool(
    name="YouTube",
    func= YouTubeSearchTool().run,
    description=(
        "Search YouTube for video tutorials, tech talks, or educational resources "
        "related to AI, data, and cloud technologies."
    ),
)

# ---- Combine Tools ----
tools = [wiki_tool, youtube_tool]

# ---- Initialize the AI Agent ----
agent = initialize_agent(
    tools=tools,
    llm=chat_model,
    agent_type="zero-shot-react-description",
    verbose=True
)

# ---- Interactive Prompt ----
print("👋 Hello! What topic are you exploring today?")
theme = input("Enter a topic: ")

try:
    query = f"Summarize {theme} from Wikipedia and list 2 YouTube videos."
    if query:
        response = agent.invoke({"input": query})
        print("\n📘 Here's what I found:\n")
        print(response["output"])
except Exception as e:
    print(f"⚠️ Error: {e}")

/home/spark-4c235d2f-cabe-4a55-9d35-ec/.ipykernel/8269/command-8300480422093416-3474296969:29: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


👋 Hello! What topic are you exploring today?


Enter a topic:  Neural Networks



> Entering new AgentExecutor chain...
To summarize Neural Networks and list 2 YouTube videos related to the topic, I need to first gather information from Wikipedia about Neural Networks. Then, I will search YouTube for relevant videos.

Action: Wikipedia
Action Input: {'query': 'Neural Networks'}
Observation: Page: Transformer (deep learning architecture)
Summary: In deep learning, the transformer is a neural network architecture based on the multi-head attention mechanism, in which text is converted to numerical representations called tokens, and each token is converted into a vector via lookup from a word embedding table. At each layer, each token is then contextualized within the scope of the context window with other (unmasked) tokens via a parallel multi-head attention mechanism, allowing the signal for key tokens to be amplified and less important tokens to be diminished. 
Transformers have the advantage of having no recurrent units, therefore requiring less training time than